In [2]:
import pandas as pd
from pathlib import Path

DATA_FOLDER = Path('../data')

print (DATA_FOLDER.resolve())

/workspaces/LegendsOfGridiron/data


In [16]:
df_results = pd.read_csv(DATA_FOLDER/'league_outcomes.csv')
for l in df_results.League.unique():
    print (l)
    df = df_results[df_results.League == l]
    # Total the Points for and the points against by team, but also the points if they were the loser   
    winner_points = df.groupby('Winner')['Winner Pts'].sum().reset_index().rename(columns={'Winner': 'Team', 'Winner Pts': 'Total Points'})
    loser_points = df.groupby('Loser')['Loser Pts'].sum().reset_index().rename(columns={'Loser': 'Team', 'Loser Pts': 'Total Points'})
    
    # Combine the points from winners and losers
    total_points_df = pd.concat([winner_points, loser_points]).groupby('Team').sum().reset_index().sort_values(by='Total Points', ascending=False)
    total_points_df['Total Points'] = round(total_points_df['Total Points'],2)
    # Now get the number of times each team has won in the league
    df_wins = df.groupby('Winner').size().reset_index(name='Wins')
    # Now get the number of times each team has lost in the league
    df_losses = df.groupby('Loser').size().reset_index(name='Losses')
    # combine the two dataframes
    df_wins = df_wins.set_index('Winner')
    df_losses = df_losses.set_index('Loser')
    df_wins = df_wins.join(df_losses, how='outer')
    # Join the total points
    df_wins = df_wins.join(total_points_df.set_index('Team'), how='outer')
    # fill in the missing values
    df_wins = df_wins.fillna(0)
    # calculate the win percentage
    df_wins['Win_Percentage'] = round(df_wins['Wins'] / (df_wins['Wins'] + df_wins['Losses']),2)
    # Sort by win percentage
    df_wins = df_wins.sort_values('Win_Percentage', ascending=False)
    # filter out "MEDIAN"
    df_wins = df_wins[df_wins.index != 'MEDIAN']
    
    df_wins.to_csv(DATA_FOLDER/f'standings/league_{l}.csv', index=True, index_label='Team')
    

MainLeague
BB1
BB2
QuickDraft 1
Quickdraft 2
BB3
BB4
BB5


In [44]:
rankings = pd.read_csv(DATA_FOLDER / 'full_ranking.csv')

rankings.set_index('Player Fullname',inplace=True)
rankings.head()

,Rank,Position,Team,Value
Player Fullname,,,,
Patrick Mahomes,1,QB1,KCC,9999
Justin Jefferson,2,WR1,MIN,9901
Josh Allen,3,QB2,BUF,9817
C.J. Stroud,4,QB3,HOU,9159
Lamar Jackson,5,QB4,BAL,9069


In [45]:
combined_ranking = pd.merge(draft_picks, rankings, left_on='Player Fullname', right_index=True)


In [47]:
combined_ranking['differential'] = combined_ranking['pick_no'] - combined_ranking['Rank']
combined_with_diff = combined_ranking.groupby('picked_by').sum('differential')
combined_ranking

,round,roster_id,player_id,picked_by,pick_no,is_keeper,draft_slot,draft_id,years_exp,team,...,news_updated,last_name,injury_status,first_name,Player Fullname,Rank,Position,Team,Value,differential
0,1,3,4984,987074330058113024,1,NaN,1,1060410180510253056,6,BUF,...,1710201638878,Allen,NaN,Josh,Josh Allen,3,QB2,BUF,9817,-2
1,1,12,9758,998692352560746496,2,NaN,2,1060410180510253056,1,HOU,...,1707446127618,Stroud,NaN,C.J.,C.J. Stroud,4,QB3,HOU,9159,-2
2,1,1,4046,1079146414086705152,3,NaN,3,1060410180510253056,7,KC,...,1710288050761,Mahomes,NaN,Patrick,Patrick Mahomes,1,QB1,KCC,9999,2
3,1,8,6904,564369169909952512,4,NaN,4,1060410180510253056,4,PHI,...,1705419607849,Hurts,NaN,Jalen,Jalen Hurts,6,QB5,PHI,8790,-2
4,1,5,4881,735703182587600896,5,NaN,5,1060410180510253056,6,BAL,...,1710277550396,Jackson,NaN,Lamar,Lamar Jackson,5,QB4,BAL,9069,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
308,26,2,4993,564663138659733504,309,NaN,9,1060410180510253056,6,CIN,...,1710257105680,Gesicki,NaN,Mike,Mike Gesicki,235,TE28,CIN,2193,74
309,26,9,5955,857665738834558976,310,NaN,10,1060410180510253056,5,NaN,...,1710346555671,Renfrow,NaN,Hunter,Hunter Renfrow,251,WR100,FA,2012,59
314,27,9,9479,857665738834558976,315,NaN,10,1060410180510253056,1,PIT,...,1705095921731,Washington,NaN,Darnell,Darnell Washington,228,TE27,PIT,2236,87
318,27,7,9506,992197862165377024,319,NaN,6,1060410180510253056,1,TB,...,1706768102404,Tucker,NaN,Sean,Sean Tucker,215,RB66,TBB,2425,104


In [35]:
# Read the users.json file into a pandas dataframe
users = pd.read_json(DATA_FOLDER/'users.json')
users

,user_id,settings,metadata,league_id,is_owner,is_bot,display_name,avatar
0,560240846271045632,NaN,"{'mention_pn': 'on', 'allow_pn': 'on'}",1060410179696533504,0.0,False,jnappy,67206d81d03c9ee856a4e7391699a24e
1,564369169909952512,NaN,"{'team_name': 'Hurtsquake', 'mention_pn': 'on'...",1060410179696533504,1.0,False,ProfessorD,f116b3ea246bae9412ffe56dba55bba8
2,564663138659733504,NaN,"{'team_name': 'Malice in the Midwest', 'mentio...",1060410179696533504,NaN,False,gpeisert1,b90085b8fce696503d0e83e00136659a
3,735703182587600896,NaN,"{'team_name': 'Keon Keepin’ On', 'mention_pn':...",1060410179696533504,NaN,False,Mschmoopie,85cf599b3c1799a2da12bfae56da6c97
4,857665738834558976,NaN,"{'mention_pn': 'on', 'allow_pn': 'on'}",1060410179696533504,NaN,False,aledo,f0edbf4278f53f9425db175073df6584
5,987074330058113024,NaN,"{'team_name': 'West Coast Nonsense ', 'mention...",1060410179696533504,NaN,False,WestCoastNonsense,550d132d7263404942564f5b81b696f5
6,990442792889405440,NaN,"{'team_name': 'the Bandit', 'mention_pn': 'on'...",1060410179696533504,NaN,False,theBandit216,4f4090e5e9c3941414db40a871e3e909
7,991952313524797440,NaN,"{'mention_pn': 'on', 'allow_pn': 'on'}",1060410179696533504,NaN,False,plfinken,8eb8f8bf999945d523f2c4033f70473e
8,992197862165377024,NaN,"{'team_name': 'Waterboys', 'mention_pn': 'on',...",1060410179696533504,NaN,False,tford51,e7af4deab0289b4f5505646424895246
9,992249754476257280,NaN,"{'team_name': 'Cactus Hack', 'mention_pn': 'on...",1060410179696533504,NaN,False,CoryPeisert,b319fdf8b7b5b0359d3c78622ba4d70c


In [41]:
merged_df = users.merge(combined_with_diff, left_on='user_id', right_index=True)[['display_name', 'differential']]
merged_df.sort_values('differential')
# Join users dataframe to the combined_with_differential, join on user_id, picked_by and return display_name and differential   


,display_name,differential
5,WestCoastNonsense,-26
10,koolaid403,21
9,CoryPeisert,90
1,ProfessorD,159
6,theBandit216,170
12,ConorDeLa,175
2,gpeisert1,206
3,Mschmoopie,215
11,ChiJV,239
4,aledo,450


In [61]:
smallest_differential = combined_ranking.groupby('picked_by')['differential'].min().reset_index()
result = smallest_differential.merge(combined_ranking[['picked_by', 'differential', 'Player Fullname']], on=['picked_by', 'differential'])
# # result = result[['differential', 'picked_by', 'Player Fullname']]
#result
users.merge(result, left_on='user_id', right_on='picked_by')[['display_name', 'differential']].sort_values('differential')

# Find the biggest reach by finding the min differential by picked_by

#smallest_differential

,display_name,differential
5,theBandit216,-142
10,ChiJV,-93
4,WestCoastNonsense,-83
8,CoryPeisert,-76
0,ProfessorD,-67
2,Mschmoopie,-61
9,koolaid403,-46
11,ConorDeLa,-42
1,gpeisert1,-38
3,aledo,-33


In [ ]:
def get_transaction(transaction_id):
    return {"txn_id": transaction_id, "txn_date": "2020-01-01", "txn_amount": 1000}

# Defining a transaction as a dictionary
transaction = {"txn_id": 1, "txn_date": "2020-01-01", "txn_amount": 1000}

# Defining a transaction using a class
transaction_2 = Transaction()
transaction_2.txn_id=1
transaction_2.txn_date="2020-01-01"
transaction_2.txn_amount=1000

# Accessing the transaction data as a dictionary
txn_id = transaction['txn_id']
txn_date = transaction['txn_date']
txn_amount = transaction['txn_amount']

# Accessing the transaction data as a class
txn2_id = transaction_2.txn_id
txn2_date = transaction_2.txn_date
txn2_amount = transaction_2.txn_amount


    